In [ ]:
day_obs = 20241017

# Facilities Temperatures Reports

In [ ]:
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time
import warnings

from argparse import Namespace
from astropy.time import Time, TimeDelta
from scipy.fft import fft, fftfreq
from scipy.signal import detrend, get_window, welch
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData, getDayObsStartTime
from lsst_efd_client.efd_helper import merge_packed_time_series


# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

# Create an EFD client
client = makeEfdClient()

# Create a folder for plots
os.makedirs("./plots", exist_ok=True)

# Constants used in the notebook
ess_weather_station_sal_index = 301

## Query the data

In [ ]:
start_time = getDayObsStartTime(day_obs)
end_time = getDayObsStartTime(day_obs + 1)

print(
    f"\nQuery data for {day_obs}"
    f"\n  starts at {start_time.isot} and"
    f"\n  ends at {end_time.isot}\n"
)

The cell bellow contains data that comes from the Weather Station Tower.

In [ ]:
df_outside = getEfdData(
    client=client,
    topic="lsst.sal.ESS.temperature",
    columns=["temperatureItem0", "salIndex"],
    begin=start_time,
    end=end_time,
)

# Select the data from the weather station using the salIndex
mask = df_outside.salIndex == ess_weather_station_sal_index
df_outside = df_outside[mask]

# We do not need the salIndex anymore
df_outside = df_outside.drop(columns=['salIndex'])

# Get the rolling min/mean/max values for the temperature
df_outside = df_outside.rename(columns={"temperatureItem0": "temperature"})
df_outside = df_outside.resample("1min").agg(
    {"temperature": ["min", "mean", "max"]}
)
df_outside.columns = df_outside.columns.droplevel(0)

MTMount contains multiple temperature sensors near the telescope.  
For the sake of simplicity, we will monitor the temperature sensors near the 
Top End Assembly for now.

In [ ]:
df_inside = getEfdData(
    client=client,
    topic="lsst.sal.MTMount.topEndChiller",
    columns=["ambientTemperature"],
    begin=start_time,
    end=end_time,
)

# Get the rolling min/mean/max values for the temperature
df_inside = df_inside.rename(columns={"ambientTemperature": "temperature"})
df_inside = df_inside.resample('1T').agg({
    'temperature': ['mean', 'min', 'max']
})
df_inside.columns = df_inside.columns.droplevel(0)

(add more information about Glycol cold)

In [ ]:
df_glycol_cold = getEfdData(
    client=client,
    topic="lsst.sal.MTMount.cooling",
    columns=["glycolTemperaturePier0101"],
    begin=start_time,
    end=end_time,
)

df_glycol_cold = df_glycol_cold.rename(columns={"glycolTemperaturePier0101": "temperature"})
df_glycol_cold = df_glycol_cold.resample('1T').agg({
    'temperature': ['mean', 'min', 'max']
})
df_glycol_cold.columns = df_glycol_cold.columns.droplevel(0)

(add more information about Glycol General)

In [ ]:
df_glycol_general = getEfdData(
    client=client,
    topic="lsst.sal.MTMount.generalPurposeGlycolWater",
    columns=["glycolTemperaturePier0001"],
    begin=start_time,
    end=end_time,
)

# Get the rolling min/mean/max values for the temperature
df_glycol_general = df_glycol_general.rename(columns={"glycolTemperaturePier0001": "temperature"})
df_glycol_general = df_glycol_general.resample('1T').agg({
    'temperature': ['mean', 'min', 'max']
})
df_glycol_general.columns = df_glycol_general.columns.droplevel(0)

## Temperatures Plots 

For each plot we have a solid line representing the rolling average per minute. 
In addition, the figure below contains a shaded region per color showing the min/max 
values per telemetry. However, since the min/max values do not stray far from the 
average, they are almost imperceptible.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_outside.index, df_outside["mean"], label='Outside Temp', color='cornflowerblue')
ax.fill_between(df_outside.index, df_outside["min"], df_outside["max"], color='cornflowerblue', alpha=0.5)

ax.plot(df_inside.index, df_inside["mean"], label='Inside Temp', color='darkorange')
ax.fill_between(df_inside.index, df_inside["min"], df_inside["max"], color='darkorange', alpha=0.5)

ax.plot(df_glycol_cold.index, df_glycol_cold["mean"], label='Glycol Cold Temp', color='green')
ax.fill_between(df_glycol_cold.index, df_glycol_cold["min"], df_glycol_cold["max"], color='green', alpha=0.5)

ax.plot(df_glycol_general.index, df_glycol_general["mean"], label='Glycol General Temp', color='red')
ax.fill_between(df_glycol_general.index, df_glycol_general["min"], df_glycol_general["max"], color='red', alpha=0.5)

fig.suptitle(f"Temperature for {day_obs}")
fig.autofmt_xdate()

ax.set_xlabel('Time [UTC]')
ax.set_ylabel('Temperature (ºC)')
ax.set_title("Temperature at the Outside, Inside, and Glycol Water")

ax.legend()
ax.grid(":", alpha=0.25)
ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))

plt.savefig(f"./plots/temperature_outside_{day_obs}.png")
plt.show()

## Hourly statistics

Here we have a table per topic with the min/mean/max value every hour. 

In [ ]:
# Resample each data frame to hourly frequency
df_outside_hourly = df_outside.resample('H').nearest()

# Change the format of the index to include only year, month, day, hour, and minute
df_outside_hourly.index = df_outside_hourly.index.strftime('%Y-%m-%d %H:%M')

print(df_outside_hourly)

In [ ]:
# Resample each data frame to hourly frequency
df_inside_hourly = df_inside.resample('H').nearest()

# Change the format of the index to include only year, month, day, hour, and minute
df_inside_hourly.index = df_inside_hourly.index.strftime('%Y-%m-%d %H:%M')

# Resample each data frame to hourly frequency
print(df_inside_hourly)

In [ ]:
# Resample each data frame to hourly frequency
df_glycol_cold_hourly = df_glycol_cold.resample('H').nearest()

# Change the format of the index to include only year, month, day, hour, and minute
df_glycol_cold_hourly.index = df_glycol_cold_hourly.index.strftime('%Y-%m-%d %H:%M')

print(df_glycol_cold_hourly)

In [ ]:
# Resample each data frame to hourly frequency
df_glycol_general_hourly = df_glycol_general.resample('H').nearest()

# Change the format of the index to include only year, month, day, hour, and minute
df_glycol_general_hourly.index = df_glycol_general_hourly.index.strftime('%Y-%m-%d %H:%M')

print(df_glycol_general_hourly)